In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import time
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, StackingClassifier

In [3]:
# Function to extract features and target variables

def x_y(df):
    y = df['0']
    x = df.drop(columns=['0'])
    return x, y

In [ ]:
# Function to extract and load dataset, returns a tuple of 2 dataframes, x and y for csv passed in argument

def loader(file_name):
    s = time.time()
    

In [4]:
s1 = time.time()
bio_mx_tr = pd.read_csv('11_Bio_Cat2_Max_training.csv')
s2 = time.time()
bio_mx_vl = pd.read_csv('11_Bio_Cat2_Max_validation.csv')
s3 = time.time()
bio_mx_ev = pd.read_csv('11_Bio_Cat2_Max_evaluation.csv')
s4 = time.time()
print('Training dataset loading time {:6.4f} s\n'.format(s2-s1))
print('Validation dataset loading time {:6.4f} s\n'.format(s3-s2))
print('Evaluation dataset loading time {:6.4f} s\n'.format(s4-s3))

Training dataset loading time 1.5194 s

Validation dataset loading time 0.2910 s

Evaluation dataset loading time 1.1857 s



In [5]:
x_tr, y_tr = x_y(bio_mx_tr)
x_vl, y_vl = x_y(bio_mx_vl)
x_ev, y_ev = x_y(bio_mx_ev)

In [6]:
# Function to train and evaluate model on dataset
import threading
print_lock = threading.Lock()

def scorer(model, x_ts, y_ts, name = 'Model'):
    print(' {} \n Score: {:6.4f}'.format(name, model.score(x_ts, y_ts)))

def train(model, x_tr, y_tr, x_vl, y_vl, x_ev, y_ev, name='Model'):
    st = time.time()
    model.fit(x_tr, y_tr)
    with print_lock:
        print('Training time: {:6.4f} s\n'.format(time.time()-st))
        scorer(mod_gs, x_vl, y_vl, '{} : Cross-Validation Score:'.format(name))
        scorer(mod_gs, x_ev, y_ev, '{} : Evaluation Set Score:'.format(name))
    return model


In [7]:
# Function to train with GridSearch

def train_gs(mod, params, x_tr, y_tr, x_vl, y_vl, x_ev, y_ev, cvl, name='Model'):
    start = time.time()
    mod_gs = GridSearchCV(mod, params, cv=cvl)
    mod_gs.fit(x_tr, y_tr)
    with print_lock:
        print('Training Time : {:6.4f} seconds'.format(time.time()-start))
        
        print('\nOptimal parameter:\n', mod_gs.best_params_)
        mod_fin = mod_gs.best_estimator_
        
        scorer(mod_gs, x_vl, y_vl, '{} : Cross-Validation Score:'.format(name))
        scorer(mod_gs, x_ev, y_ev, '{} : Evaluation Set Score:'.format(name))
        
    return mod_fin

In [8]:
import multiprocessing as mp
print("No. of CPus :", mp.cpu_count())

No. of CPus : 4


In [9]:
# Function to pickle a trained model to an output.pickle file
import pickle

def pkl_out(model, name='Model'):
    op_ = open("Pkls/{}.pickle".format(name), "wb")
    pickle.dump(model, op_)
    op_.close()
    print('Success')
    
# Function to extract a trained model from a pickle file

def pkl_in(name):
    ip_ = open("Pkls/{}.pickle".format(name), "rb")
    mod = pickle.load(ip_)
    return mod


In [11]:

class trainer(threading.Thread):
    
    
    def __init__(self, model, params, name='Model'):
        threading.Thread.__init__(self)
        self.model = model
        self.params = params
        self.name = name
        
    def run(self):
        train_gs(self.model, self.params, x_tr, y_tr, x_vl, y_vl, x_ev, y_ev, 5, self.name)
        

In [12]:
# A) Serialized Implementation

# Similar to the previous dataset, we take 4 individual learners in the following order: LogReg, KNN, Random Forest, and SVC. 

s = time.time()

# 1. LogReg
lr = LogisticRegression()
params_lr = {'max_iter':[300, 500, 750],
             'C':[0.80, 1.0, 1.2]}
lr_f = train_gs(lr, params_lr, x_tr, y_tr, x_vl, y_vl, x_ev, y_ev, 5, 'Logistic Regression')


# 2. KNN
knn = KNeighborsClassifier()
params_knn = {'n_neighbors': np.arange(1,25)}
knn_f = train_gs(knn, params_knn, x_tr, y_tr, x_vl, y_vl, x_ev, y_ev, 5, 'K Nearest Neighbors')


# 3. Random Forest
rf = RandomForestClassifier()
params_rf = {'n_estimators':[10, 50, 100, 150, 200, 250]}
rf_f = train_gs(rf, params_rf, x_tr, y_tr, x_vl, y_vl, x_ev, y_ev, 5, 'Random Forest')


# 4. SVC
sv = SVC()
params_sv = {'max_iter':[300, 400, 500, 600, 750]}
sv_f = train_gs(sv, params_sv, x_tr, y_tr, x_vl, y_vl, x_ev, y_ev, 5, 'Support Vector')


keys = ['lr', 'knn', 'rf', 'sv']
models = [('lr', lr_f), ('knn', knn_f), ('rf', rf_f), ('sv', sv_f)]

print('Overall SeriaL Training Time : {:6.4f} s'.format(time.time()-s))


C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regre

Training Time : 31.6091 seconds

Optimal parameter:
 {'max_iter': 300}
 Logistic Regression : Cross-Validation Score: 
 Score: 0.9938
 Logistic Regression : Evaluation Set Score: 
 Score: 0.8121
Training Time : 373.3930 seconds

Optimal parameter:
 {'n_neighbors': 1}
 K Nearest Neighbors : Cross-Validation Score: 
 Score: 0.9187
 K Nearest Neighbors : Evaluation Set Score: 
 Score: 0.7414
Training Time : 106.4212 seconds

Optimal parameter:
 {'n_estimators': 100}
 Random Forest : Cross-Validation Score: 
 Score: 0.9938
 Random Forest : Evaluation Set Score: 
 Score: 0.6886


C:\Users\User\Anaconda3\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=300).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=300).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=300).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=300).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\User\Anaconda3\lib\site-packages\sk

Training Time : 58.0573 seconds

Optimal parameter:
 {'max_iter': 300}
 Support Vector : Cross-Validation Score: 
 Score: 0.9938
 Support Vector : Evaluation Set Score: 
 Score: 0.7993
Overall SeriaL Training Time : 583.8789 s


In [ ]:
pkl_out(lr, 'LogReg_serial')
pkl_out(knn, 'KNearNbr_serial')
pkl_out(rf, 'RanFor_serial')
pkl_out(svc, 'SupVec_serial')

In [13]:
# B) Threaded Implementation
lr = LogisticRegression()
knn = KNeighborsClassifier()
rf = RandomForestClassifier()
sv = SVC()

stock_models = {'lr': lr,
                'knn': knn,
                'rf': rf, 
                'sv': sv}
param_dict = {'lr': params_lr,
              'knn': params_knn, 
              'rf': params_rf, 
              'sv': params_sv}

s = time.time()

# for i in keys:
#     t = threading.Thread(target=trainer, args=(stock_models[i], param_dict[i], x_tr, y_tr, x_vl, y_vl, x_ev, y_ev, 5, i))
#     t.start()

threads = []
for i in keys:
    thread = trainer(stock_models[i], param_dict[i], i)
    threads += [thread]
    thread.start()

for x in threads:
    x.join()
    
print('Overall Threaded Training Time : {:6.4f} s'.format(time.time()-s))    


C:\Users\User\Anaconda3\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=300).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=300).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceW

Training Time : 103.2668 seconds

Optimal parameter:
 {'max_iter': 300}
 lr : Cross-Validation Score: 
 Score: 0.9938
 lr : Evaluation Set Score: 
 Score: 0.8121


C:\Users\User\Anaconda3\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=300).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Training Time : 116.1574 seconds

Optimal parameter:
 {'max_iter': 300}
 sv : Cross-Validation Score: 
 Score: 0.9938
 sv : Evaluation Set Score: 
 Score: 0.7993
Training Time : 219.0256 seconds

Optimal parameter:
 {'n_estimators': 150}
 rf : Cross-Validation Score: 
 Score: 0.9875
 rf : Evaluation Set Score: 
 Score: 0.7121
Training Time : 410.7997 seconds

Optimal parameter:
 {'n_neighbors': 1}
 knn : Cross-Validation Score: 
 Score: 0.9187
 knn : Evaluation Set Score: 
 Score: 0.7414
Overall Threaded Training Time : 421.7223 s


In [ ]:
pkl_out(lr, 'LogReg_parallel')
pkl_out(knn, 'KNearNbr_parallel')
pkl_out(rf, 'RanFor_parallel')
pkl_out(svc, 'SupVec_parallel')